In [1]:
!pip install --quiet transformers

     |████████████████████████████████| 2.3MB 6.9MB/s 
     |████████████████████████████████| 3.3MB 37.5MB/s 
     |████████████████████████████████| 901kB 50.0MB/s 


Mount Google Drive and copy reddit sentiment csv files if running inside Colab. Put reddit comment files to `data/reddit` manually if running locally

In [2]:
!mkdir -p data/reddit
!mkdir -p sentiments
!cp drive/MyDrive/nlp/redditComments/* data/reddit

In [5]:
!cp drive/MyDrive/nlp/deps/RobertaSentiment.py .

In [6]:
import pandas as pd
import numpy as np
from RobertaSentiment import RobertaSentimentModel

roberta = RobertaSentimentModel()

In [7]:
tickers = ["AMC", "GME", "NOK", "PLTR", "TSLA", "AAPL"]
sentiment = []

print("Sentimenting comments...")
for stock in tickers:
    print(f"...{stock}")
    df = pd.read_csv(f"data/reddit/comments-{stock}.csv")

    df["comment_created"] = pd.to_datetime(df.comment_created, unit="s")
    df["comment_created"] = df.comment_created.dt.round("D")
    df = df[df.comment_created >= "2021-01-01"]
    df = df[df.comment_created < "2021-05-01"]

    dates = df.comment_created.sort_values().unique()
    for date in dates:
        commentsMatrix = df[df.comment_created == date].comment_body.to_numpy()
        commentsMatrix = commentsMatrix[:100]
        for i,txt in enumerate(commentsMatrix):
          #XXX make pytorch not fail. remove unicode shenanigans and keep the texts short
          txt = txt.encode("ASCII", "ignore")
          txt = txt[:512]
          commentsMatrix[i] = txt


        scores = roberta.score(commentsMatrix)
        mean_total_score = np.mean([score["total_score"] for score in scores])
        sentiment.append({"Date": date, "Stock": stock, "Sentiment": mean_total_score})


print("All done. Saving to csv")
df = pd.DataFrame(data=sentiment)
df.to_csv("sentiments/reddit-roberta-sentiment.csv")

Sentimenting comments...
...AMC
...GME
...NOK
...PLTR
...TSLA
...AAPL
All done. Saving to csv


In [8]:
!cp sentiments/reddit-roberta-sentiment.csv drive/MyDrive/nlp/sentiments